In [171]:
from unittest.mock import inplace

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization
import matplotlib.pyplot as plt
import re
%matplotlib inline
pd.set_option('future.no_silent_downcasting', True)
pd.options.mode.copy_on_write = True

data = pd.DataFrame( pd.read_excel('./breast-cancer.xls') )  # Data Import from local/ Drive file


################# Data sanitisation #################
total_rows = len(data)
print("Data size before sanitisation, Total: ", total_rows)
data.dropna(inplace=True) # Drops all NA cells by row and saves it back to source data


print("Removed, Rows with NA: ", total_rows - len(data))
for x in data.index:

    if re.search( r"^[0-9]{1,2}-[0-9]{1,2}$",  str(data.loc[x, "age"]) ) is None and not ( re.search( r"^[0-9]{1,2}$",  str(data.loc[x, "age"])is None) ): #Using RegEx to confirm age pattern is not "XX-XX" / "X-XX" and not "XX"
        data.drop(index=x, inplace=True)
    elif re.search(r"^(ge40)$|^(premeno)$|^(lt40)$", str(data.loc[x, "menopause"])) is None:  #Using RegEx to confirm menopause name is only 1 of three
        data.drop(index=x, inplace=True)
    elif re.search(r"^[0-9]{1,2}-[0-9]{1,2}$", str(data.loc[x, "tumor-size"]) ) is None:  #Using RegEx to confirm tumor size is formated correctly
        data.drop(index=x, inplace=True)
    elif re.search(r"^[0-9]{1,2}-[0-9]{1,2}$", str(data.loc[x, "inv-nodes"])) is None: #Using RegEx to confirm node-caps is only yes or no Upper or lower case
        data.drop(index=x, inplace=True)
    elif re.match(r'^([Yy][eE][Ss]|[Nn][oO])$', str(data.loc[x, "node-caps"])) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(index=x, inplace=True)
    elif re.search(r"^1$|^2$|^3$", str(data.loc[x, "deg-malig"])) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(index=x, inplace=True)
    elif re.search(r"^([Ll][eE][fF][tT]|[Rr][Ii][Gg][hH][tT])$", str(data.loc[x, "breast"])) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(index=x)
    elif re.search(r"^(left_up|left_low|central|right_up|right_low)$", str(data.loc[x, "breast-quad"]), re.IGNORECASE) is None:
        data.drop(index=x)#Using RegEx to confirm inv-nodes count is a number range

    #Force only standard answer for this query, Yes / Recurence will = Recurrence-event, anything with no / non = Non-recurrence-event
    elif re.search(r"(non|no|recurrence|Yes)", str(data.loc[x, "Class"]), re.IGNORECASE) is None:
        data.loc[x, "Class"] = "Non-recurrence-event"
        if re.search(r"(non|no)", str(data.loc[x, "Class"]), re.IGNORECASE) :
            data.loc[x, "Class"] = "Non-recurrence-event"
        else:
            data.loc[x, "Class"] = "Recurrence-event"
        data.drop(index=x)

data.reset_index(drop=True, inplace=True)  # Re indexes entire dataframe to fix deleted rows
print("Total removal rows: " , total_rows - len(data))
################# Data sanitisation #################

################# Data Formating #################
def range_median(column):
    main_array = []
    for y in data.index:
        value = str(data.loc[y, column])
        if re.search(r"^[0-9]{1,2}-[0-9]{1,2}$", str(value)) is not None:
            array = np.array(re.findall(r"[0-9]{1,2}", value), dtype=int)
            main_array.append(np.median(array))

        elif  re.search( r"^[0-9]{1,2}$",  value)is not None:
            array = np.array(re.findall(r"[0-9]{1,2}", value), dtype=int)
            main_array.append(np.median(array))
    data[str(column)] = main_array
    return main_array


def uniques_translator(column, inpalce_trigger):
    column_uniques = np.array(pd.unique(data[column]))
    translation_dictionary = {}
    for x in range(column_uniques.size):
        translation_dictionary.update({x : column_uniques[x]})
    if inpalce_trigger == True:
        data[str(column)] = data[[str(column)]].replace(column_uniques, translation_dictionary.keys())
        return data[[str(column)]]
    else:
        return data[[str(column)]].replace(column_uniques, translation_dictionary) , translation_dictionary

##  Dictionary of the change to revert back later    ##


numeric_Translation_Menopause = uniques_translator('menopause', False)
numeric_Translation_NodeCaps = uniques_translator('node-caps', False)
numeric_Translation_Breast = uniques_translator('breast', False)
numeric_Translation_irradiat = uniques_translator('irradiat', False)
numeric_Translation_Class = uniques_translator('Class', False)

##  Dictionary of the change to revert back later    ##

## Translating all string columns to int equivilant  ##

uniques_translator('menopause', True)
uniques_translator('node-caps', True)
uniques_translator('breast-quad', True)
uniques_translator('breast', True)
uniques_translator('irradiat', True)
uniques_translator('Class', True)
## Translating all string columns to int equivilant  ##
#range_median('age')


## Range medium finder for all non int only columns
range_median('age')
range_median('tumor-size')
range_median('inv-nodes')


################# Data Formating #################

################# Training Data Masking #################
training_data_per =  90/100.0

data['train'] = np.random.rand(len(data)) < training_data_per # Creates random array for each row  of Data, true or false using the fraction
Training_Data = data[data.train == 1] # Appends train Data with the array for each row, effectively showing which row at random will be removed

Training_Data = Training_Data.drop('train', axis=1).sample(frac=1)
################# Training Data Masking #################

################# Testing Data Seperation / Masking #################
Testing_Data = data[data.train == 0]
Testing_Data.drop('train', axis=1, inplace=True)

Training_Data = data[data.train == 1]
Training_Data.drop('train', axis=1, inplace=True)
#Using X to remove Class colum as Y will be the opposite, As it is the diagnoses column
#Y = 0  1
#    1  0
#    1  0
#    1  0
#    0  1
# == [0,0,0,1.....]
X = Training_Data.drop('Class', axis=1).to_numpy(dtype=float)

targets = [[1,0],[0,1]]
Y = np.array([targets[int(z[0])]  for z in Training_Data.values[:,8:9]])

################# Testing Data Seperation / Masking #################

################# Backpropagating data creation #################

NumberInput = len(X[0])
Hidden_layer_nueron = 9
#print(np.random.default_rng(seed=4))
weight = 2*np.random.random((NumberInput, Hidden_layer_nueron)) - 1


NumberOutputs = len(Y[0])
weight2 = 2*np.random.random((Hidden_layer_nueron, NumberOutputs)) - 1

################# Backpropagating data creation #################

################# Backpropagating data creation #################

Data size before sanitisation, Total:  286
Removed, Rows with NA:  0
Total removal rows:  99


array([[-0.95409513, -0.79453598, -0.32402816,  0.81688163,  0.96481716,
        -0.51499266,  0.49479091, -0.06390836,  0.97849242],
       [ 0.75585663, -0.43461786, -0.72306453,  0.00873409,  0.60308463,
        -0.09306249,  0.90395361,  0.35572612, -0.96039447],
       [ 0.11204924,  0.96135697, -0.20991014, -0.63693681,  0.75694931,
        -0.12032514,  0.63408457, -0.95989777, -0.9307018 ],
       [ 0.29285068, -0.25952526, -0.80990044, -0.73027794, -0.17193681,
         0.9486073 , -0.81836725,  0.09737782,  0.79723771],
       [-0.07144002, -0.20632279, -0.70130435, -0.673975  ,  0.48171833,
        -0.46067462,  0.37311719, -0.73837067, -0.01660664],
       [-0.58360595,  0.04198017, -0.46684705,  0.96024212, -0.02938926,
         0.26530903, -0.39945541,  0.55108696, -0.51723633],
       [-0.02586638,  0.33491939,  0.99468107, -0.84116473,  0.37683379,
        -0.91758727, -0.1104861 , -0.76021987, -0.57691988],
       [-0.03357184,  0.95497877,  0.89708488,  0.19229137,  0